# Purpose: Dahl Mglur5 Microglia OGD Cell Analysis

### Purpose: To quantify cell features of already segmented microglia images from step 1 in this folder

Created by: Hawley Helmbrecht

Creation Date: 06/4/2021 (Original file)

Edit Date: 10/27/2021 - Change the file to be for Phuongs BeV Analysis rather than the Dahl Mglur5 Project
6/16/21 - Updated for the Mglur5 analysis

*Step 1: Import Necessary Packages*

In [1]:
import numpy as np
import pandas as pd
from scipy import ndimage

import skimage.filters
from skimage import morphology
from skimage.measure import label, regionprops, regionprops_table
from skimage.color import label2rgb
from skimage import io
from skimage import measure
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from skimage import segmentation

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

import watermark
import os
from PIL import Image

*Step 2: User Inputs*

In [2]:
#replace the example path from my computer with the path to the image on your computer

cell_folder = '/Users/hhelmbre/Desktop/Mglur5_publication_analysis/mean_thresh_array'

file_type = '.npy'

*Step 3: Defining a Folder Cleaner Function to only Return Tif Images*

In [3]:
def folder_cleaner(folder, image_type):
    k=0
    for files in folder:
        if image_type in str(files):
            k+=1
        else:
            folder = np.delete(folder, np.argwhere(folder == str(files)))
    return folder

*Step 4: Get All Images in the Folder*

In [4]:
arr = os.listdir(cell_folder)
file_list = np.asarray(arr)
file_list = folder_cleaner(file_list, file_type)

In [5]:
file_list

array(['351_slide5_slice4_CA2_60x_zstack5_mean_thresh.npy',
       '351_slide5_slice3_CA1_60x_zstack3_mean_thresh.npy',
       '351_slide5_slice5_cortex_60x_zstack1_mean_thresh.npy',
       '238_slide4_slice3_CA1_60x_zstack1_mean_thresh.npy',
       '309_slide5_slice5_CA2_60x_zstack4_mean_thresh.npy',
       '352_slide2_slice4_cortex_60x_zstack4_mean_thresh.npy',
       '363_slide5_slice3_CA3_60x_zstack2_mean_thresh.npy',
       '363_slide5_slice3_cortex_60x_zstack1_mean_thresh.npy',
       '308_slide5_slice5_cortex_60x_zstack2_mean_thresh.npy',
       '352_slide2_slice3_CA2_60x_zstack5_mean_thresh.npy',
       '343_slide7_slice4_cortex_60x_zstack3_mean_thresh.npy',
       '352_slide2_slice4_CA1_60x_zstack3_mean_thresh.npy',
       '309_slide5_slice5_CA1_60x_zstack1_mean_thresh.npy',
       '213_slide5_slice3_CA2_60x_zstack5_mean_thresh.npy',
       '238_slide4_slice4_CA1_60x_zstack2_mean_thresh.npy',
       '337_slide5_slice5_CA1_60x_zstack3_mean_thresh.npy',
       '238_slide4_slice3

*Step 5: Segmenting and Calculating Region Features on All Images*

In [6]:
properties_list = ('area', 'bbox_area', 'centroid', 'convex_area', 
                   'eccentricity', 'equivalent_diameter', 'euler_number', 
                   'extent', 'filled_area', 'major_axis_length', 
                   'minor_axis_length', 'orientation', 'perimeter', 'solidity')


In [72]:
# Without watershed Separation

In [25]:
j = 0
source_dir = '/Users/hhelmbre/Desktop/Mglur5_publication_analysis/'

for names in file_list:
        file_name = str(cell_folder + '/' + names)
        new_binary_mean = np.load(file_name)
        label_image = label(new_binary_mean)
        
        #Need to add in a separation step here
        
        #Feel free to add them here as well. The computational time is pretty efficient
        props = measure.regionprops_table(label_image, properties=(properties_list))

        if j == 0:
            df = pd.DataFrame(props)
            df['filename'] = names
        else:
            df2 = pd.DataFrame(props)
            df2['filename'] = names
            df = df.append(df2)
        
        #Add in saving the segmentation labeled images
        short_im_name = file_name.split('/')[6]
        plt.imsave(str(source_dir + 'labeled_no_watershed/' + short_im_name[:-4] + '_labled_no_watershed.png'), label_image, cmap='gist_ncar')
        
        j = 1

*Step 6: Caculating the Circularity*

In [8]:
df['circularity'] = 4*np.pi*df.area/df.perimeter**2

*Step 7: Calculating the Aspect Ratio*

In [9]:
df['aspect_ratio'] = df.major_axis_length/df.minor_axis_length

*Step 8: Add in a column for the ID*

In [10]:
df

,area,bbox_area,centroid-0,centroid-1,convex_area,eccentricity,equivalent_diameter,euler_number,extent,filled_area,major_axis_length,minor_axis_length,orientation,perimeter,solidity,filename,circularity,aspect_ratio
0,1527,7569,97.700720,416.055010,5027,0.729961,44.093500,1,0.201744,1527,83.866168,57.321547,0.906161,706.434596,0.303760,351_slide5_slice4_CA2_60x_zstack5_mean_thresh.npy,0.038451,1.463083
1,787,5226,104.641677,274.062262,2340,0.960241,31.655008,1,0.150593,787,91.078059,25.426525,-0.661207,360.841666,0.336325,351_slide5_slice4_CA2_60x_zstack5_mean_thresh.npy,0.075954,3.582010
2,1390,6972,157.058993,170.057554,4463,0.739342,42.069026,1,0.199369,1390,83.772056,56.406222,1.196556,622.949314,0.311450,351_slide5_slice4_CA2_60x_zstack5_mean_thresh.npy,0.045011,1.485156
3,372,1782,147.997312,84.104839,1201,0.880263,21.763389,1,0.208754,372,56.022787,26.582037,-0.047663,241.988853,0.309742,351_slide5_slice4_CA2_60x_zstack5_mean_thresh.npy,0.079829,2.107543
4,453,2279,172.068433,421.953642,1320,0.820055,24.016193,1,0.198771,453,52.769646,30.199286,-0.910768,228.409163,0.343182,351_slide5_slice4_CA2_60x_zstack5_mean_thresh.npy,0.109114,1.747381
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16,409,2816,354.955990,71.567237,1585,0.884928,22.820056,1,0.145241,409,62.924354,29.305605,-0.243054,280.409163,0.258044,308_slide5_slice4_CA1_60x_zstack1_mean_thresh.npy,0.065366,2.147178
17,1024,4992,374.013672,463.912109,4359,0.855675,36.108133,1,0.205128,1024,99.418895,51.450589,0.072551,627.694480,0.234916,308_slide5_slice4_CA1_60x_zstack1_mean_thresh.npy,0.032660,1.932318
18,1068,5980,391.202247,187.601124,4169,0.740145,36.875735,1,0.178595,1068,85.746813,57.660242,0.045904,629.357431,0.256177,308_slide5_slice4_CA1_60x_zstack1_mean_thresh.npy,0.033883,1.487105
19,583,2025,415.178388,95.101201,1285,0.909368,27.245158,1,0.287901,583,58.018751,24.135385,-0.832226,253.273124,0.453696,308_slide5_slice4_CA1_60x_zstack1_mean_thresh.npy,0.114209,2.403888


*Step 10: Saving as a CSV file*

In [11]:
df.to_csv('/Users/hhelmbre/Desktop/Mglur5_publication_analysis/mean_threshold_quantified_cells_no_watershed.csv')

# With Watershed Separation

In [24]:
j = 0
df=[]
source_dir = '/Users/hhelmbre/Desktop/Mglur5_publication_analysis/'

for names in file_list:
        file_name = str(cell_folder + '/' + names)
        new_binary_mean = np.load(file_name)
        label_image = label(new_binary_mean)
        
        #Need to add in a separation step here
        distance = ndimage.distance_transform_edt(label_image)
        coords = peak_local_max(distance, footprint=np.ones((265, 265)), labels=label_image)
        mask = np.zeros(distance.shape, dtype=bool)
        mask[tuple(coords.T)] = True
        markers, _ = ndimage.label(mask)
        labels = watershed(-distance, markers, mask=label_image)
        
        #Feel free to add them here as well. The computational time is pretty efficient
        props = measure.regionprops_table(labels, properties=(properties_list))

        if j == 0:
            df = pd.DataFrame(props)
            df['filename'] = names
        else:
            df2 = pd.DataFrame(props)
            df2['filename'] = names
            df = df.append(df2)
        
        #Add in saving the segmentation labeled images
        short_im_name = file_name.split('/')[6]
        plt.imsave(str(source_dir + 'labeled_with_watershed/' + short_im_name[:-4] + '_labled_with_watershed.png'), labels, cmap='gist_ncar')
        
        j = 1

In [22]:
#Circularity
df['circularity'] = 4*np.pi*df.area/df.perimeter**2

#Aspect Ratio
df['aspect_ratio'] = df.major_axis_length/df.minor_axis_length

#viewing df
df

,area,bbox_area,centroid-0,centroid-1,convex_area,eccentricity,equivalent_diameter,euler_number,extent,filled_area,major_axis_length,minor_axis_length,orientation,perimeter,solidity,filename,circularity,aspect_ratio
0,1527,7569,97.700720,416.055010,5027,0.729961,44.093500,1,0.201744,1527,83.866168,57.321547,0.906161,706.434596,0.303760,351_slide5_slice4_CA2_60x_zstack5_mean_thresh.npy,0.038451,1.463083
1,787,5226,104.641677,274.062262,2340,0.960241,31.655008,1,0.150593,787,91.078059,25.426525,-0.661207,360.841666,0.336325,351_slide5_slice4_CA2_60x_zstack5_mean_thresh.npy,0.075954,3.582010
2,372,1782,147.997312,84.104839,1201,0.880263,21.763389,1,0.208754,372,56.022787,26.582037,-0.047663,241.988853,0.309742,351_slide5_slice4_CA2_60x_zstack5_mean_thresh.npy,0.079829,2.107543
3,1390,6972,157.058993,170.057554,4463,0.739342,42.069026,1,0.199369,1390,83.772056,56.406222,1.196556,622.949314,0.311450,351_slide5_slice4_CA2_60x_zstack5_mean_thresh.npy,0.045011,1.485156
4,453,2279,172.068433,421.953642,1320,0.820055,24.016193,1,0.198771,453,52.769646,30.199286,-0.910768,228.409163,0.343182,351_slide5_slice4_CA2_60x_zstack5_mean_thresh.npy,0.109114,1.747381
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22,127,350,347.905512,481.283465,234,0.917210,12.716187,1,0.362857,127,25.165358,10.025984,-0.301919,76.905592,0.542735,308_slide5_slice4_CA1_60x_zstack1_mean_thresh.npy,0.269835,2.510014
23,897,4896,377.710145,461.452620,3655,0.901493,33.794909,1,0.183211,897,99.091986,42.886437,0.232832,565.788889,0.245417,308_slide5_slice4_CA1_60x_zstack1_mean_thresh.npy,0.035212,2.310567
24,1068,5980,391.202247,187.601124,4169,0.740145,36.875735,1,0.178595,1068,85.746813,57.660242,0.045904,629.357431,0.256177,308_slide5_slice4_CA1_60x_zstack1_mean_thresh.npy,0.033883,1.487105
25,583,2025,415.178388,95.101201,1285,0.909368,27.245158,1,0.287901,583,58.018751,24.135385,-0.832226,253.273124,0.453696,308_slide5_slice4_CA1_60x_zstack1_mean_thresh.npy,0.114209,2.403888


In [23]:
df.to_csv('/Users/hhelmbre/Desktop/Mglur5_publication_analysis/mean_threshold_quantified_cells_with_watershed.csv')

*Step 11: Print Dependencies and State*

In [15]:
%load_ext watermark

%watermark -v -m -p numpy,pandas,scipy,skimage,matplotlib,wget

%watermark -u -n -t -z

Python implementation: CPython
Python version       : 3.7.4
IPython version      : 7.8.0

numpy     : 1.21.5
pandas    : 1.3.5
scipy     : 1.3.1
skimage   : 0.17.2
matplotlib: 3.1.1
wget      : 3.2

Compiler    : Clang 4.0.1 (tags/RELEASE_401/final)
OS          : Darwin
Release     : 20.6.0
Machine     : x86_64
Processor   : i386
CPU cores   : 8
Architecture: 64bit

Last updated: Thu Jun 16 2022 13:00:16PDT

